# Patent Reduction

## Purpose

The purpose of this program is to identify the core similiarities between 42 patents and applications, and reduce their number to the top 3 or 4 based on ngrams and other factors.

## Work Plan

- Get full text of patents
- Analyze ngrams of pantents
- Identify embeddings 
- Conduct dimension reduction technique

In [95]:
import pandas as pd
from bs4 import BeautifulSoup
from bs4 import SoupStrainer
from urllib.request import urlopen
import re

In [96]:
patent_folder ='./Patents/gp-search-20170622-225739.csv'

In [97]:
df = pd.read_csv(patent_folder, skiprows=1)

In [137]:
df

,id,title,assignee,inventor/author,priority date,filing/creation date,publication date,grant date,result link
0,US7286245B2,Method and apparatus for determining the influ...,Carl Zeiss Smt Ag,"Ulrich Wegmann, Michael Hartl, Markus Mengel, ...",2002-07-29,2003-07-29,2007-10-23,2007-10-23,https://patents.google.com/patent/US7286245B2/en
1,US20080043236A1,Optical Property Measurement Apparatus and Opt...,Nikon Corporation,"Koji Kaise, Toru Fujii, Yasushi Mizuno",2004-08-09,2005-08-09,2008-02-21,NaN,https://patents.google.com/patent/US2008004323...
2,US7277182B2,Apparatus for polarization-specific examinatio...,Carl Zeiss Smt Ag,"Ulrich Wegmann, Markus Mengel",2003-07-05,2004-07-06,2007-10-02,2007-10-02,https://patents.google.com/patent/US7277182B2/en
3,US6788389B2,Production method of projection optical system,Nikon Corporation,"Youhei Fujishima, Hironori Ikezawa, Toshihiko ...",2001-07-10,2002-07-10,2004-09-07,2004-09-07,https://patents.google.com/patent/US6788389B2/en
4,US6870668B2,Method for evaluating image formation performa...,Nikon Corporation,Toshihiko Ozawa,2000-10-10,2001-10-09,2005-03-22,2005-03-22,https://patents.google.com/patent/US6870668B2/en
5,US20060119838A1,Optical measuring apparatus and operating meth...,Carl Zeiss Smt Ag,"Wolfgang Emer, Ulrich Wegmann, Martin Schrieve...",2003-05-12,2005-11-14,2006-06-08,NaN,https://patents.google.com/patent/US2006011983...
6,US20050190376A1,Device and method for the determination of ima...,Carl Zeiss Smt Ag,"Ulrich Wegmann, Helmut Haidner, Gordon Doering",2004-01-21,2005-01-21,2005-09-01,NaN,https://patents.google.com/patent/US2005019037...
7,US7298498B2,Optical property measuring apparatus and optic...,Nikon Corporation,Akira Takahashi,2003-10-16,2006-04-14,2007-11-20,2007-11-20,https://patents.google.com/patent/US7298498B2/en
8,US20060001861A1,Measuring method and measuring system for meas...,Carl Zeiss Smt Ag,Ulrich Wegmann,2002-12-19,2005-06-20,2006-01-05,NaN,https://patents.google.com/patent/US2006000186...
9,US20100020302A1,Projection exposure tool for microlithography ...,Carl Zeiss Smt Ag,Rolf Freimann,2007-01-23,2009-07-22,2010-01-28,NaN,https://patents.google.com/patent/US2010002030...


In order to download the text of each of the patents listed above, we'll use the unique urls for each patent provided by the United States Patent and Trademark Office and then scrape the pages using BeautifulSoup. 

## Generating the Unique URLs

Select the "id" column in the data frame. 

In [99]:
patent_id_column=df['id']

Next, create a list that contains each id number

In [100]:
patent_id = [i for i in patent_id_column]

An inspection of the USPTO website reveals that the url for each patent contains the patent number repeated three times within the url. However, the initial 'US' and final 'A1' or 'B2' has been removed. 

To to that end, we need to slice each item in the patent_id list so as to remove the intial two and final two charachers from each item.   

In [101]:
patent_id_numbers =[i[2:-2] for i in patent_id_column]

For example:

In [102]:
patent_id[0] 

'US7286245B2'

versus

In [103]:
patent_id_numbers[0]

'7286245'

Next, we take the patent_id_numbers and insert them into the url template so as to create the unique url for each patent.

To do that, we'll use a for loop where we replace each instance of the patent number with 'i'.

In [165]:
url_list = []
for i in patent_id_numbers:
    #Use each of the unique patent id numbers to create a unique url
    url = "http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO1&Sect2=HITOFF&d=PALL&p=1&u=%2Fnetahtml%2FPTO%2Fsrchnum.htm&r=1&f=G&l=50&s1="+i+".PN.&OS=PN/"+i+"&RS=PN/"+i
    url_list.append(url)

Now we have a list of the unique urls for each of the patents. 

Next, in order to analyze the text, we need to to get the text from patent's webpage. To do so, we use BeautifulSoup.  